In [4]:
!pip -q install redis redis-cli

In [ ]:
import redis
r = redis.Redis(host="127.0.0.1", port=6379, db=0)


## Improve your Redis developer experience with RedisInsight, Redis Labs - [Link to Video](https://youtu.be/ppYSS6opUiQ)

### Overview

RedisInsight is a desktop GUI for interacting with Redis via a data browser, integrated CLI, command profiler, database analysis, slowlog viewer, bulk key operations, and more. It supports Windows, macOS, Linux, and Docker. Feature references: [Browser](https://redis.io/docs/latest/develop/tools/insight/#browser), [Profiler](https://redis.io/docs/latest/develop/tools/insight/#profiler), [CLI](https://redis.io/docs/latest/develop/tools/insight/#cli), [Database analysis](https://redis.io/docs/latest/develop/tools/insight/#database-analysis), [Streams support](https://redis.io/docs/latest/develop/tools/insight/#redis-streams-support), [Bulk actions](https://redis.io/docs/latest/develop/tools/insight/#bulk-actions), [Slow Log](https://redis.io/docs/latest/develop/tools/insight/#slow-log), [Telemetry](https://redis.io/docs/latest/develop/tools/insight/#telemetry), [Platform notes / logs](https://redis.io/docs/latest/develop/tools/insight/#log-files).  

Supported Redis distributions and services referenced in the session include Redis Open Source, Redis Enterprise, Redis Cloud, and major managed providers.

### Recent Updates (as stated in the session)

- RedisInsight 1.10 released.  
- Added support for Redis modules referenced in the talk: RedisJSON, RediSearch 2.0, RedisGears; RedisGraph and RedisTimeSeries were also mentioned. Module docs: [RedisJSON](https://redis.io/docs/latest/operate/oss_and_stack/stack-with-enterprise/json/), [RediSearch 2.x release notes](https://redis.io/docs/latest/operate/oss_and_stack/stack-with-enterprise/release-notes/redisearch/redisearch-2.0-release-notes/), [RedisGears v1](https://redis.io/docs/latest/operate/oss_and_stack/stack-with-enterprise/gears-v1/), [RedisGraph (EOL notice)](https://redis.io/docs/latest/operate/oss_and_stack/stack-with-enterprise/deprecated-features/graph/), [RedisTimeSeries](https://github.com/RedisTimeSeries/RedisTimeSeries).
- CLI enhancements: integrated command help and command repeat; configurable key bindings.  
- Browser improvements: resizing, display, and filtering fixes.  
- Telemetry system introduced (anonymous, opt-in).

### Telemetry Snapshot (as stated in the session)

- Participation: ~41% opted in.  
- Usage distribution observed: ~62% Browser, ~23% CLI; notable usage of modules and Streams; remaining features ~1% or less.  
- Telemetry is anonymous and used to guide prioritization. See [RedisInsight telemetry](https://redis.io/docs/latest/develop/tools/insight/#telemetry).

### User Survey Highlights (as stated in the session)

- Priorities ranked: ~45% prioritize easy browsing/interaction with data; ~31% prefer OS‑native desktop use alongside IDE; ~14% want direct integration; ~8% prefer IDE plugin.

### Roadmap Focus (as stated in the session)

- RedisInsight v2 emphasis on developer experience: intuitive navigation (clarity, consistency, accessibility, discoverability), increased developer productivity, and in‑app guidance. Public preview targeted for August (beta earlier).

### Demo Context

Goal: inspect how a Node.js queuing framework (Bull) leverages Redis data structures, using RedisInsight to visualize queue state transitions for producer/consumer flows and to clear jobs in specific states.

### Queue Lifecycle (Bull on Redis)

Bull jobs move through states such as `wait` (queued), `delayed`, `active`, `completed`, and `failed`. Internally, Redis data structures back these states (lists for waiting, sorted sets for delayed/completed/failed, hashes for job payload/metadata, and strings for counters/ephemeral keys). See Bull’s lifecycle overview and state semantics: [Bull Guide – Job Lifecycle](https://optimalbits.github.io/bull/#a-job-s-lifecycle), [BullMQ Architecture (states & sets)](https://docs.bullmq.io/guide/architecture).

> **Sidenote**  
>  
> [Link: Bull — “A Job’s Lifecycle”](https://optimalbits.github.io/bull/#a-job-s-lifecycle)  
>  
> **Concept**: `Bull job lifecycle` → Jobs transition from `wait`/`delayed` → `active` → `completed` or `failed`; stalled detection can retry or fail jobs.  
>  
> **Context**: The session demonstrates producing three jobs (simple, delayed, intentionally failing) and observing their transitions in RedisInsight.  
>  
> **Example**: A delayed job sits in a sorted set until its timestamp elapses, then moves to the waiting list for processing.  
>  
> **Implication**: State sets/lists and job hashes enable introspection and tooling (e.g., clearing delayed jobs or verifying completions).

### Redis Data Structures Observed

- **List**: queue of waiting job IDs (FIFO). See [Lists](https://redis.io/docs/latest/develop/data-types/lists/).  
- **Sorted set (zset)**: timestamps or scores track delayed, completed, and failed jobs. See [Sorted sets](https://redis.io/docs/latest/develop/data-types/sorted-sets/).  
- **Hash**: per‑job payload and metadata. See [Hashes](https://redis.io/docs/latest/develop/data-types/hashes/).  
- **String**: counters/ephemeral markers (e.g., last job ID, stall checks). See [Data types overview](https://redis.io/docs/latest/develop/data-types/).

### Producer Step 1 — Add a Basic Job and Inspect Waiting List

```python
# Append a job ID to the right (tail) of the waiting list (modeled as a FIFO queue)
r.execute_command("RPUSH", "job_queue:wait", "1")

# Verify: check list length and contents
r.execute_command("LLEN", "job_queue:wait")           # expect >= 1
print(r.execute_command("LRANGE", "job_queue:wait", "0", "-1"))  # expect ['1', ...]
````

> **Sidenote**
>
> [Link: RPUSH](https://redis.io/docs/latest/commands/rpush/)
>
> **Command**: `RPUSH` → Push element(s) to the tail of a list.
>
> **Pattern**: `RPUSH key element [element ...]`
>
> **Example**: `RPUSH job_queue:wait 1 2 3`
>
> **Result**:
>
> Returns the list length after push; creates the list if absent.

> **Sidenote**
>
> [Link: LRANGE](https://redis.io/docs/latest/commands/lrange/)
>
> **Command**: `LRANGE` → Retrieve a range of list elements by index.
>
> **Pattern**: `LRANGE key start stop`
>
> **Example**: `LRANGE job_queue:wait 0 -1`
>
> **Result**:
>
> Array of elements within the specified index range.

> **Sidenote**
>
> [Link: LLEN](https://redis.io/docs/latest/commands/llen/)
>
> **Command**: `LLEN` → Get list length.
>
> **Pattern**: `LLEN key`
>
> **Example**: `LLEN job_queue:wait`
>
> **Result**:
>
> Integer count of elements.

### Producer Step 1 — Persist Job Body as a Hash

```python
# Store job payload/metadata in a hash keyed by job ID
r.execute_command("HSET", "job:1", "process", "true", "delay", "false")

# Verify: retrieve all fields/values for the job hash
print(r.execute_command("HGETALL", "job:1"))  # expect ['process','true','delay','false',...]
```

> **Sidenote**
>
> [Link: HSET](https://redis.io/docs/latest/commands/hset/)
>
> **Command**: `HSET` → Set one or more field‑value pairs in a hash.
>
> **Pattern**: `HSET key field value [field value ...]`
>
> **Example**: `HSET job:1 process true delay false`
>
> **Result**:
>
> Number of fields added (updated fields overwrite values).

> **Sidenote**
>
> [Link: HGETALL](https://redis.io/docs/latest/commands/hgetall/)
>
> **Command**: `HGETALL` → Get all fields and values in a hash.
>
> **Pattern**: `HGETALL key`
>
> **Example**: `HGETALL job:1`
>
> **Result**:
>
> Flat array of field/value pairs.

### Producer Step 1 — Track Last Job ID with a String

```python
# Update a counter or "last id" marker (frameworks often maintain an incrementing job id)
r.execute_command("SET", "job_queue:last_id", "1")

# Verify: read the last job id string
print(r.execute_command("GET", "job_queue:last_id"))  # expect '1'
```

> **Sidenote**
>
> [Link: GET](https://redis.io/docs/latest/commands/get/)
>
> **Command**: `GET` → Get the string value of a key.
>
> **Pattern**: `GET key`
>
> **Example**: `GET job_queue:last_id`
>
> **Result**:
>
> String value or nil if the key does not exist.

### Producer Step 2 — Add a Delayed Job

```python
# Insert job id 2 into a "delayed" sorted set with a score representing the ready-at timestamp
r.execute_command("ZADD", "job_queue:delayed", "1699999999", "2")

# Verify: list delayed jobs with scores
print(r.execute_command("ZRANGE", "job_queue:delayed", "0", "-1", "WITHSCORES"))  # expect ['2','1699999999', ...]
```

> **Sidenote**
>
> [Link: ZADD](https://redis.io/docs/latest/commands/zadd/)
>
> **Command**: `ZADD` → Add member(s) with score(s) to a sorted set.
>
> **Pattern**: `ZADD key score member [score member ...]`
>
> **Example**: `ZADD job_queue:delayed 1699999999 2`
>
> **Result**:
>
> Count of new elements added; updates score if member exists.

> **Sidenote**
>
> [Link: ZRANGE](https://redis.io/docs/latest/commands/zrange/)
>
> **Command**: `ZRANGE` → Return sorted-set members in a range, optionally with scores.
>
> **Pattern**: `ZRANGE key start stop [WITHSCORES]`
>
> **Example**: `ZRANGE job_queue:delayed 0 -1 WITHSCORES`
>
> **Result**:
>
> Members in index range; includes scores when requested.

### Producer Step 3 — Add a Job Configured to Fail During Processing

```python
# Append failing job id 3 to the waiting list and store metadata indicating "fail on process"
r.execute_command("RPUSH", "job_queue:wait", "3")
r.execute_command("HSET", "job:3", "process", "true", "shouldFail", "true")

# Verify: queue contains job 1 and 3; job:3 hash reflects failure flag
print(r.execute_command("LRANGE", "job_queue:wait", "0", "-1"))  # expect ['1','3', ...]
print(r.execute_command("HGETALL", "job:3"))                     # expect ['shouldFail','true', ...]
```

### Consumer — Process Jobs and Observe State Sets

After running the consumer, three sorted sets are present for state tracking: `job_queue:completed`, `job_queue:delayed`, and `job_queue:failed`. Verify membership:

```python
# Verify sets after processing: job 1 completed, job 2 still delayed, job 3 failed
print(r.execute_command("ZRANGE", "job_queue:completed", "0", "-1"))  # expect ['1', ...]
print(r.execute_command("ZRANGE", "job_queue:delayed", "0", "-1"))    # expect ['2', ...]
print(r.execute_command("ZRANGE", "job_queue:failed", "0", "-1"))     # expect ['3', ...]
```

> **Sidenote**
>
> [Link: LPOP](https://redis.io/docs/latest/commands/lpop/), [Link: BRPOPLPUSH](https://redis.io/docs/latest/commands/brpoplpush/), [Link: BLMOVE](https://redis.io/docs/latest/commands/blmove/)
>
> **Command**: `LPOP` / `BRPOPLPUSH` / `BLMOVE` → Commands used by queue workers to atomically pop/move jobs between lists (active/processing) with optional blocking semantics.
>
> **Pattern**:
> `LPOP key [count]`
> `BRPOPLPUSH source destination timeout`
> `BLMOVE source destination <LEFT|RIGHT> <LEFT|RIGHT> timeout`
>
> **Example**: `BLMOVE job_queue:wait job_queue:active LEFT RIGHT 0`
>
> **Result**:
>
> Atomically removes an element from the source and pushes it to the destination (blocking variant waits for new jobs). See command docs for semantics and time complexity.

### Maintenance — Clear Jobs in a Specific State (Delayed)

Option A: remove a specific delayed job and delete its hash.

```python
# Remove job id 2 from the delayed set and delete its hash
r.execute_command("ZREM", "job_queue:delayed", "2")
r.execute_command("DEL", "job:2")

# Verify: job 2 no longer present; its hash key is gone
print(r.execute_command("ZRANGE", "job_queue:delayed", "0", "-1"))  # expect not to include '2'
print(r.execute_command("EXISTS", "job:2"))                         # expect 0
```

Option B (as demonstrated): delete the entire delayed set if appropriate.

```python
# Delete the delayed set entirely
r.execute_command("DEL", "job_queue:delayed")

# Verify: key removed and type no longer a sorted set
print(r.execute_command("EXISTS", "job_queue:delayed"))  # expect 0
```

> **Sidenote**
>
> [Link: ZREM](https://redis.io/docs/latest/commands/zrem/)
>
> **Command**: `ZREM` → Remove one or more members from a sorted set.
>
> **Pattern**: `ZREM key member [member ...]`
>
> **Example**: `ZREM job_queue:delayed 2`
>
> **Result**:
>
> Integer count of members removed; if last member is removed, the key is deleted.

> **Sidenote**
>
> [Link: DEL](https://redis.io/docs/latest/commands/del/)
>
> **Command**: `DEL` → Remove one or more keys.
>
> **Pattern**: `DEL key [key ...]`
>
> **Example**: `DEL job_queue:delayed job:2`
>
> **Result**:
>
> Integer count of keys removed.

> **Sidenote**
>
> [Link: EXISTS](https://redis.io/docs/latest/commands/exists/) · [Link: TYPE](https://redis.io/docs/latest/commands/type/)
>
> **Command**: `EXISTS` / `TYPE` → Check key existence and infer value type.
>
> **Pattern**: `EXISTS key [key ...]`, `TYPE key`
>
> **Example**: `EXISTS job:2`, `TYPE job_queue:delayed`
>
> **Result**:
>
> `EXISTS` returns the number of existing keys; `TYPE` returns the data type string.

### RedisInsight Features Used in the Demo

* **Data Browser**: listed keys with type-aware indicators; quickly inspected list/zset/hash/string content. See [Browser](https://redis.io/docs/latest/develop/tools/insight/#browser).
* **CLI / Workbench**: executed commands, used in-app assistance. See [CLI](https://redis.io/docs/latest/develop/tools/insight/#cli), [Workbench](https://redis.io/docs/latest/develop/tools/insight/#workbench).
* **Slow Log & Profiler**: tools for performance analysis; Slow Log surfaces commands exceeding a threshold (backed by Redis `SLOWLOG`). See [Slow Log tool](https://redis.io/docs/latest/develop/tools/insight/#slow-log), [SLOWLOG command](https://redis.io/docs/latest/commands/slowlog/), [Profiler](https://redis.io/docs/latest/develop/tools/insight/#profiler).
* **Database Analysis**: memory/key distribution insights. See [Database analysis](https://redis.io/docs/latest/develop/tools/insight/#database-analysis).

> **Sidenote**
>
> [Link: SLOWLOG](https://redis.io/docs/latest/commands/slowlog/)
>
> **Concept**: `Redis Slow Log` → Records commands whose execution time exceeds a configured threshold.
>
> **Context**: RedisInsight’s Slow Log tool visualizes these entries for troubleshooting.
>
> **Example**: `SLOWLOG GET` retrieves entries; `SLOWLOG RESET` clears the log.
>
> **Implication**:
>
> Identifies slow commands for optimization without client I/O time.

> **Sidenote**
>
> [Link: MEMORY (container command)](https://redis.io/docs/latest/commands/memory/) · [Link: MEMORY USAGE](https://redis.io/docs/latest/commands/memory-usage/)
>
> **Concept**: `Memory introspection` → Inspect memory usage per key and aggregated stats.
>
> **Context**: Database analysis in RedisInsight leverages Redis memory reporting to surface distribution and heavy keys.
>
> **Example**: `MEMORY USAGE job:1` estimates bytes consumed.
>
> **Implication**:
>
> Enables targeted memory optimization and cleanup.

### Data Type Context (Used by Queue Frameworks)

> **Sidenote**
>
> [Link: Lists](https://redis.io/docs/latest/develop/data-types/lists/)
>
> **Concept**: `List` → Linked list of strings commonly used for queues.
>
> **Context**: Waiting queue of job IDs.
>
> **Example**: `RPUSH job_queue:wait 1` then `LPOP job_queue:wait`.
>
> **Implication**:
>
> Natural FIFO semantics for job dispatch.

> **Sidenote**
>
> [Link: Sorted sets](https://redis.io/docs/latest/develop/data-types/sorted-sets/)
>
> **Concept**: `Sorted set` → Unique members ordered by score (e.g., timestamps).
>
> **Context**: Delayed scheduling and completed/failed tracking.
>
> **Example**: `ZADD job_queue:delayed 1699999999 2`.
>
> **Implication**:
>
> Efficient scheduling/ordering and range queries.

> **Sidenote**
>
> [Link: Hashes](https://redis.io/docs/latest/develop/data-types/hashes/)
>
> **Concept**: `Hash` → Field–value map to store structured metadata.
>
> **Context**: Per‑job payload/flags (e.g., `process=true`, `shouldFail=true`).
>
> **Example**: `HSET job:3 shouldFail true`.
>
> **Implication**:
>
> Compact, mutable representation for job records.

### Notes on Streams (mentioned in usage metrics)

Redis Streams are an append-only log with consumer groups and random access operations, suitable for event processing. See [Streams overview](https://redis.io/docs/latest/develop/data-types/streams/).

```python
# Example: append an event to a stream and read it (illustrative; not used by Bull)
r.execute_command("XADD", "events", "*", "type", "job", "id", "1")
print(r.execute_command("XRANGE", "events", "-", "+"))  # verify entries exist
```



### Original Transcript

[Music] hi everyone and welcome to the session where we'll be talking about radice insight and how you can improve your redis developer experience so my name is olga lopaj and i'm a product manager for redis inside right so the agenda for today will include a quick recap of what redis insight is what's the latest in the world of radius insert what are our plans for future and we will conclude with a short demo at the end right so let's dive in so what is redis  inside redis insight is a is a visual tool for developers that empowers them to better use redis today so that's just a short description but what does it really mean in actual terms technical terms so redis insight consists of a number of features or capabilities and these include a data browser that allows you to quickly interact and manipulate your data in redis there's a cli which lets you run your commands directly against your redis environment it has support for a number of redis modules these include redis json uh radius j um ready search redis graph redis time series and radius gears it also has a memory analysis and that's exactly what it says in the tin it it runs an analysis of of redis memory and it will provide you information useful information you know in case you need to troubleshoot or optimize your environment further this is slow log analysis but we'll point out the slow running queries there's also a profiler which will give you an overview of your commands so for example such metrics as how many commands have been run top commands how many clients are connected and also there's a bulk actions feature which lets you prepare your commands and run them in bulk without the need so the commands to do data manipulations will valve the need to write that uh to put that down into a script all right so all of that allows you to quite efficiently visualize interact debug troubleshoot learn and discover all things redis related so redis insight also operates on windows mac os and linux environments and is also available as a docker image as mentioned it will uh work with redis open source redis enterprise redis in the cloud and also the majority of of redis providers as a service as well right so what's the latest with redis insight we've recently released redis version 1.10 so go and please check it out if you haven't yet also since last conference we've we've made some improvements to redis inside but here are some some of the highlights so we introduced support for redis json uh pardon uh support for redis search 2.0 support for reddisk gears and support for radice's radius ai is coming out very very soon um also uh there's improvements we've we've done some improvements to the cli so we've added a command help and a command repeat where you're able to repeat some of the commands you've previously run you also you can now configure key bindings in terms of browser we've we've done some fixes to resizing display and and filtering we also uh quite excitedly introduced a new telemetry system and that basically the the aim of that is to allow us to collect enough data to make informed decisions as to where we need to focus our attention um you know during the next um during the time during during the next versions of redis inside right so um telemetry um to note about we have quite a good participation rate about 41 which is which is really good and based on the uh data we've collected to know that the data is all of the data is anonymous and so we we only look at we try to understand where our user base so redis inside users are spending most of their time so and here you can see the breakdown so we've noticed that 62 understandably is of your activity is within the browser uh 23 within cli the substantial activity with redis graph redis search already and streams um the rest of the features uh they're being used to a lesser extent so about one percent or less than that right so the question is is is this representative of your you know day-to-day behavior because not would really like to hear um to to understand uh you know and the other side as well if this is not representative so if it's so please opt in to share telemetry either way whether that represent your behavior or not uh please do um opt in because we'd like to hear you know from even more um than just 41 percent of our user base right we also run a user survey it's still running where we've asked users to prioritize the four areas um meaning which ones of those matter most to them so forty percent and forty five percent uh mentioned that uh easy browsing interacting and interacting with data is the most important thing in redis inside then having redis inside as an os native application to run locally along with their ide so 31 mentioned that's the most important thing to them 14 mentioned that they would like you know redis inside directly integrated redis and eight percent said that red is inside uh they'd like to see redis inside as an ide plugin so right so obviously uh this these results help us understand where we should be focusing next which brings me to the next uh slide so what does the future hold for redis inside so logically it's a red is inside v2 but with a very specific focus which will be on developer experience optimization so what does that mean in actual term so we'll be focusing on three main areas so that means we'd like to bring an intuitive navigation and that means that we will be building the next version of redis inside with clarity consistency accessibility and discoverability in mind so that when you're in one part of the application you actually know what you you know what your next step will be you're not lost you don't you don't need to go and keep on checking you know um you don't need to go on checking google what to do next so right so and then it's increased developer productivity uh so we aim to you know decrease that that time to market um metric uh for developers as to and that means you know a shorter development a cycle also a very important and very dear um area to me is the in-app guidance we understand the importance of of quick onboarding a quick of good documentation and the ability to easily interact learn experiment and understand you know redis you know going just beyond just the cash use case in in redis all right so red is inside v2  we aim to have a public preview coming in august but obviously we will have a beta release earlier than that so if you'd like to participate in in this beta program please sign up here by following this link here so we would welcome you and we'll contact you so please please sign up all right so let's dive into a demo so for this demo let's assume the role of someone who needs to implement a queuing system in a microservice oriented architecture and we decide to leverage redis because it's fast it's it's quite um it's a quite fast keystore um uh library so let's so we will leveraging we'll be leveraging redis but we also would like to obfuscate the um uh the low level details and the complexities so we'll be uh we've chosen a wrapper to run on top of that which will be the bull package uh um for queuing system it's a queuing system and um obviously before we even go into implementing a design or proper developing we'd like to explore how um you know how this framework will work so that we you know have enough information to make decision design decisions at a later stage and also we're thinking because bool and redis were open source we'd like perhaps in the future we'll if we implement you know some um uh features on top of of of bull for example we would like to know the internal workings of it so we'll be using redis inside in order to deconstruct how bull leverages ready's data structures but regardless of that regardless of the framework you use of the programming language whether there's a corresponding ui or not as long as you use redis you can use the redis inside to to understand and and query and um and easily interact to your already store all right to summarize what's uh what's a queuing system so a queuing system will will have at least one producer that will publish jobs to the queue and a consumer that will process those jobs off with you so uh in bull um bull manages jobs uh through a life cycle and the diagram here represents that represents that live circuit job life cycle so job once it it's added to the queue it can go through a number of states you know depending on the options configured so you can go into a waiting state delayed state uh eventually and end up in a failed or completed state as once all the retries have been exhausted all right so what we want to understand is basically how a job progresses through you know each stage so um our testing will will do exactly that so we'll look at the producer in isolation we'll look at the consumer in isolation and also we will look at a way to clear some of those jobs in particular state in case we need in the future we need to implement some sort of garbage collection all right so let's dive in right so here at the top we have our ide with free scripts so producer a consumer and a script to clear jobs at the bottom here we have our redis inside v2 in a dark theme again this is just a preview to know that some of the features planned features are still lucky but i think it's still cool up to show at this stage so it's running as a desktop a desktop application at the moment  all right so here we're presented with the list of redis databases as as configured and to be accessed from redis inside so let's go ahead and navigate to the one of interest to us so red is queue  we're presented here with uh it's empty as we haven't run anything but we're presented here with the classic you know view of on your left hand side you have your key list and on your right hand side you have your key details panel all right so let's go ahead and try and and run our first script which is a producer to note we have a few um breakpoints scattered so that we can analyze the output of a script in redis at different stages so in a producer we will we will add three different um well free jobs uh with different options into into vm uh into onto the queue okay so let's go ahead and and so basically what we're doing that is to analyze you know how each of those um jobs can add um in a in a particular state all right so let's go ahead and run um so here to note that we are creating a queue called job queue right so let's go ahead and run the next bit um okay so let's look at the debug console as mentioned here we're sending one job with just you know a very basic body no uh no options configured so let's go ahead and refresh our redis inside to see if anything has been added into the um database so yeah indeed we can see  three uh different data structures added into redis and how i know all three of them are different is because in v2 we associate a particular colored label with a particular data structure so visual visual cues they help you know you know to draw your attention to a particular um data structure of interest at that time so in in my case for example if if there's a sea of hashes i can i can find you know that list um a data structure that's of interest to me so since we are so so let's go ahead and look at these structures and understand how you know um visa leveraged in in this queuing system um  so since we associate a queue with a list let's go ahead and start with a queue here so all three of these have been created as a result of our script running because all three of them contain the job queue name as pointed earlier  if i click on this i can see there's one element in it so i can safely infer from the actual name and the element in here that that's probably a list where jobs that are in a waiting state um it's keeping track of of jobs in in a waiting state so indeed the job id number one is add has been added here there's also a string which probably points to um to the last job added onto the queue but we can confirm that um during our next run so  the hash here seems to contain the body of of a job so if we go navigate to the data we can see that yeah indeed process equal true that's her body matches the body here and debug console all right and a few more few more options in here which since we haven't provided any options to v here we could yeah we can see if a visa haven't been set to anything  right so that's good let's go ahead and run the next part of a script now we i we're sending another job but this time we've also introduced an option which is we would like to delay that that job all right so let's refresh redis inside  okay so we have two more data structures added so there's another hash added so let's go ahead so uh hash number two let's go ahead and and check that yep the body matches um yeah  it looks good process true delay true so same as here now let's go ahead and validate that indeed the assumptions about the list and the strings the string data structures are correct so the list since this is a waiting state indeed it just has just one but we can also see another sorted set created here which points to delayed state yes so indeed it has a job number two in here as instructed by our options here all right and then the string indeed it's pointing to the last job um published to a queue okay  now let's run a rest of script and we're adding another job and since we want to see the unhappy paths as well we've we've configured this job um so that intention is for it to fail at the uh when it will be consumed um uh in in the next script so we'd like to see you know uh a job that is completed we'd like to see a job that's still in a delayed state since we introduce a quite lengthy delay period and we want to see a job that is is in a failed state so the consumer will basically what we'll do we'll check this label here to process it or not all right so let's refresh again and just to verify that our third job has been added added into the into radius delayed stayed the same since we only have one job here but waiting state yep we have two elements now one pointing to id three and id one okay so let's exit this part of our testing and go to consumer script  right so uh as mentioned we expect one job to complete one job to failed and one job to stay as it is currently in a delayed state so let's go ahead and run it and refresh  and let's see what's going to happen okay so there's a there's another string here a stall check has a very short ttl so probably some sort of transient data structure used internally by by the boo but we're not going to be worried too much about this at this point so we do have three now sets sorted sets and red is  that uh as from the name we can see that one is keeping track of failed job jobs one is keeping track of delayed jobs and one skipping track of completed jobs so let's go ahead and validate that well we previously assumed this correct so let's go ahead to the completed one so yeah job id uh is is part of this set delayed yep job id is still in in a delayed state and failed job id3 is in a fail state so our subjects have been confirmed  now let's go ahead now and exit this script and try and clear as mentioned before we might at some point want to clear some of these jobs in particular states so this this script will attempt to clear any jobs that are in a delayed state so that's job id number two okay so let's go ahead and run it and  refresh yep as we can see that said that was responsible for keeping track of delayed jobs has been deleted and hash with the id of two is all has also been cleared out all right so now you can see kind of the power of redis inside when you need to quickly interact with redis during your perhaps debugging developing just testing exploratory testing mode all right so  um that concludes our demo and our presentation hope you found it useful um and here are a few links that i would encourage you to follow to access more information on reddit's inside to read a bit more on the modules and also get in touch with us via discord and lastly uh don't forget to sign up for redis inside beta program alright thank you for listening to me and good luck bye



n